# Imports

In [1]:
from functools import reduce
from pathlib import Path
import time
import datetime

import pandas as pd
import requests
from wmfdata import hive,spark
from wmfdata.utils import print_err, pd_display_all
from wmfdata.utils import check_kerberos_auth, ensure_list


# Parameters

In [2]:
# TSV file where metrics are or will be saved
FILENAME = "metrics/metrics.tsv"

# Metric month. The mediawiki_history snapshot must be from the metrics month or later.
# METRICS_MONTH_TEXT = '2019-07'
# MEDIAWIKI_HISTORY_SNAPSHOT = '2019-07'

last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

METRICS_MONTH_TEXT = last_month.strftime("%Y-%m")
MEDIAWIKI_HISTORY_SNAPSHOT = last_month.strftime("%Y-%m")


# Preparation

In [3]:
# Convert our metrics month to all the formats we need and provide them in a dict
# so we can easily use them to format strings
metrics_month = pd.Period(METRICS_MONTH_TEXT)
date_params = {
    "mediawiki_history_snapshot": MEDIAWIKI_HISTORY_SNAPSHOT,
    "metrics_month": str(metrics_month),
    "metrics_month_start": str(metrics_month.start_time), 
    "metrics_month_first_day": str(metrics_month.asfreq("D", how="start")),
    "metrics_month_end": str((metrics_month + 1).start_time),
    "metrics_month_last_day": str(metrics_month.asfreq("D", how="end")),
    "api_metrics_month_first_day": metrics_month.asfreq("D", how="start").strftime("%Y%m%d"),
    "api_metrics_month_day_after": (metrics_month + 1).asfreq("D", how="start").strftime("%Y%m%d"),
    "metrics_prev_month": str(metrics_month - 1),
    "retention_cohort": str(metrics_month - 2)
}

# Load any previous results
try:
    old_metrics = (
        pd.read_csv(FILENAME, sep="\t", parse_dates = ["month"])
        .set_index("month")
    )
except FileNotFoundError:
    old_metrics = None
    
def prepare_query(filename):
    return (
        Path(filename)
        .read_text()
        .format(**date_params)
    )

# MariaDB and Hive query metrics

In [4]:
queries = {
    "active_editors": {
        "file": "queries/active_editors.sql",
        "engine": "hive"
    },
    "edits": {
        "file": "queries/edits.hql",
        "engine": "hive"
    },
    "new_editor_retention": {
        "file": "queries/new_editor_retention.hql",
         "engine": "hive"
    },
    "global_south_edits_editors": {
        "file": "queries/global_south_edits_editors.hql",
        "engine": "hive"
    },
    "mobile-heavy_edits_editors": {
        "file": "queries/mobile-heavy_edits_editors.hql",
        "engine": "hive"
    },
    "mobile-heavy_new_editor_retention": {
        "file": "queries/mobile-heavy_new_editor_retention.hql",
        "engine": "hive"
    },
    "global_south_new_editor_retention": {
        "file": "queries/global_south_new_editor_retention.hql",
        "engine": "hive"
    }
}


for key, val in queries.items():
    query = prepare_query(val["file"])
    engine = val["engine"]
    print_err("Running {} on {}...".format(key, engine))
    
    if engine == "mariadb":
        result = mariadb.run(query)
    elif engine == "hive":
        result = hive.run(query)
    else:
        raise ValueError("Unknown engine specified.") 
    
    result = result.assign(month=lambda df: pd.to_datetime(df["month"]))
    val["result"] = result

Running active_editors on hive...
Running edits on hive...
Running new_editor_retention on hive...
Running global_south_edits_editors on hive...
Running mobile-heavy_new_editor_retention on hive...
Running global_south_new_editor_retention on hive...
Running mobile-heavy_edits_editors on hive...


# Content metrics via API

In [5]:
NEW_PAGES_API = (
    # Replaces "https://wikimedia.org/api/rest_v1/metrics/" due to https://phabricator.wikimedia.org/P8605
    "http://aqs1004.eqiad.wmnet:7232/analytics.wikimedia.org/v1/" 
    "edited-pages/new/{project}/all-editor-types/{page_type}/monthly/{start}/{end}"
)

headers = {
    "User-Agent": "https://github.com/wikimedia-research/Editing-movement-metrics (bot)"
}

# Create container for results
api_results = []

def get_new_pages(
    project="all-projects",
    page_type="content",
    start=date_params["api_metrics_month_first_day"],
    end=date_params["api_metrics_month_day_after"]
):
    url = NEW_PAGES_API.format(
        project = project,
        page_type = page_type,
        start = start,
        end = end
    )
    
    r = requests.get(url, headers=headers)
    data = r.json()["items"][0]["results"]
    frame = pd.DataFrame(data)
    frame["timestamp"] = pd.to_datetime(frame["timestamp"]).dt.tz_localize(None)
    frame = frame.rename(columns={"timestamp": "month"})
    
    return frame

## Total

In [6]:
total_new = get_new_pages().rename(columns={"new_pages": "net_new_content_pages"})
api_results.append(total_new)

## Wikidata

In [7]:
new_wd = get_new_pages(
    project="wikidata.org"
).rename(columns={
    "new_pages": "net_new_Wikidata_entities"
})
api_results.append(new_wd)

## Commons

In [8]:
new_commons = get_new_pages(
    project="commons.wikimedia.org"
).rename(columns={
    "new_pages": "net_new_Commons_content_pages"
})
api_results.append(new_commons)

## Wikipedias

In [9]:
# Get a list of project URLs (each one in a 1-tuple)
wp_domains = spark.run("""
select domain_name
from canonical_data.wikis
where database_group = "wikipedia"
""", format="raw")

# Query the API for each project and append records to a list
results = []
n = len(wp_domains)

In [10]:

for idx, val in enumerate(wp_domains):
    domain = val[0]
    
    if idx % 10 == 0:
        msg = "Now on the {}th project of {} ({})"
        print_err(msg.format(idx, n, domain))
        
    frame = get_new_pages(project=domain).reset_index()
    frame["project"] = domain
    records = frame.to_dict("records")
    results.extend(records)
    
    # Sleep 20 milliseconds
    time.sleep(0.02)

# Turn the big list of records into a data frame
new_per_wp = pd.DataFrame(results)

# Sum across projects to get new Wikipedia articles per month
new_wp = new_per_wp.groupby("month").agg(
    {"new_pages": "sum"}
).rename(columns={"new_pages": "net_new_Wikipedia_articles"}).reset_index()

api_results.append(new_wp)

Now on the 0th project of 308 (aa.wikipedia.org)
Now on the 10th project of 308 (arc.wikipedia.org)
Now on the 20th project of 308 (ban.wikipedia.org)
Now on the 30th project of 308 (bjn.wikipedia.org)
Now on the 40th project of 308 (cbk-zam.wikipedia.org)
Now on the 50th project of 308 (crh.wikipedia.org)
Now on the 60th project of 308 (diq.wikipedia.org)
Now on the 70th project of 308 (et.wikipedia.org)
Now on the 80th project of 308 (frr.wikipedia.org)
Now on the 90th project of 308 (gl.wikipedia.org)
Now on the 100th project of 308 (he.wikipedia.org)
Now on the 110th project of 308 (hz.wikipedia.org)
Now on the 120th project of 308 (is.wikipedia.org)
Now on the 130th project of 308 (kbd.wikipedia.org)
Now on the 140th project of 308 (ko.wikipedia.org)
Now on the 150th project of 308 (la.wikipedia.org)
Now on the 160th project of 308 (lo.wikipedia.org)
Now on the 170th project of 308 (mh.wikipedia.org)
Now on the 180th project of 308 (mt.wikipedia.org)
Now on the 190th project of 30

In [11]:
# Strip timezones returned by API so our month columns merge nicely
for df in api_results:
    df["month"] = df["month"].dt.tz_localize(None)

# Combining and saving metrics

In [12]:
queries["edits"]["result"] = queries["edits"]["result"].rename({"data_edits": "wikidata_edits"}, axis=1)

In [13]:
# Assemble list of result dataframes
results = [val["result"] for _, val in queries.items()]
results.extend(api_results)

In [14]:
# Merge them all, assuming that the month is the only common column
new_metrics = reduce(lambda l, r: pd.merge(l, r, how="outer"), results)

# Set the month as an index so combine_first works properly
new_metrics = new_metrics.set_index("month").sort_index()

if old_metrics is None:
    metrics = new_metrics
else:
    metrics = new_metrics.combine_first(old_metrics)
    
pd_display_all(metrics.tail())

,active_editors,global_south_active_editors,global_south_edits,global_south_new_editor_retention,global_south_nonbot_edits,mobile-heavy_wiki_active_editors,mobile-heavy_wiki_edits,mobile-heavy_wiki_new_editor_retention,mobile-heavy_wiki_nonbot_edits,mobile_edits,net_new_Commons_content_pages,net_new_Wikidata_entities,net_new_Wikipedia_articles,net_new_content_pages,new_active_editors,new_editor_retention,other_nonbot_edits,returning_active_editors,revert_rate,total_edits,uploads,wikidata_edits
month,,,,,,,,,,,,,,,,,,,,,,
2019-11-01,85848.0,25692.0,4334840.0,NaN,3517279.0,4273.0,1742610.0,0.039161,479971.0,1538302.0,632849.0,3625578.0,211501.0,4651272.0,16756.0,0.076603,11076121.0,69092.0,0.060278,46923002.0,685487.0,23502409.0
2019-12-01,85239.0,26160.0,4172511.0,0.044190,3353040.0,4726.0,2137717.0,0.048892,491849.0,1621995.0,670121.0,3481249.0,195211.0,4737543.0,15785.0,0.069466,10643462.0,69454.0,0.055438,45318629.0,716420.0,21367199.0
2020-01-01,90229.0,27325.0,4524797.0,0.044979,3626029.0,5047.0,1667762.0,0.051835,586535.0,1745927.0,633007.0,3520037.0,252889.0,4818313.0,18342.0,0.053807,12043414.0,71887.0,0.067180,44748606.0,683969.0,19002423.0
2020-02-01,88232.0,20245.0,2191848.0,0.043097,1980937.0,4541.0,1052721.0,0.045172,506151.0,1616956.0,805704.0,2516834.0,255138.0,3782792.0,18043.0,0.054111,11279497.0,70189.0,0.060902,44139367.0,855590.0,18736393.0
2020-03-01,90443.0,21524.0,2802052.0,0.042328,2441182.0,5028.0,2955059.0,0.046599,702907.0,1678852.0,933836.0,2368419.0,468599.0,4041475.0,18254.0,0.071413,13262731.0,72189.0,0.054138,49822981.0,998024.0,20754272.0


In [15]:
metrics.to_csv(FILENAME, sep="\t")